In [2]:
import requests
from bs4 import BeautifulSoup
import os

In [4]:
def scrape_book(category_name, category_url):
    print(f"Scraping category: {category_name}")
    res = requests.get(category_url)
    soup = BeautifulSoup(res.text, "html.parser")
    book_links = soup.select("li.booklink a.link")

    folder_path = f"data/raw/{category_name}"
    os.makedirs(folder_path, exist_ok=True)

    for link in book_links[:25]:
        book_id = link['href'].split('/')[-1]
        txt_url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt"
        try:
            txt = requests.get(txt_url).text
            start = txt.find("*** START OF THIS PROJECT GUTENBERG EBOOK")
            end = txt.find("*** END OF THIS PROJECT GUTENBERG EBOOK")
            if start != -1 and end != -1:
                txt = txt[start:end]
            file_path = os.path.join(folder_path, f"book_{book_id}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(txt)
            print(f"Saved book {book_id} to {category_name}")
        except Exception as e:
            print(f"Failed to download book {book_id}: {e}")

In [ ]:
links = {
    "economic" : "https://www.gutenberg.org/ebooks/bookshelf/435",
    "history" : "https://www.gutenberg.org/ebooks/bookshelf/446",
    "business" : "https://www.gutenberg.org/ebooks/bookshelf/428",
    "science" : "https://www.gutenberg.org/ebooks/bookshelf/201"
}

for name, url in links.items():
    scrape_book(name, url)